In [1]:
%%writefile predictor.py

import os

import numpy as np
import joblib
import pandas as pd

class ChurnPredictor(object):

  _COLUMN_NAMES=['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling','PaymentMethod', 'MonthlyCharges', 'TotalCharges']

  def __init__(self, model):
    self._model = model

  def predict(self, instances, **kwargs):
    inputs = pd.DataFrame(data=[instances], columns=self._COLUMN_NAMES)
    outputs = self._model.predict(inputs)
    return outputs.tolist()

  @classmethod
  def from_path(cls, model_dir):
    model_path = os.path.join(model_dir, 'model.joblib')
    model = joblib.load(model_path)
    return cls(model)

Writing predictor.py


In [2]:
from predictor import ChurnPredictor
model = ChurnPredictor.from_path('.')
instance = ['7317-GGVPB', 'Male', 0, 'Yes', 'No', 71, 'Yes', 'Yes', 'Fiber optic', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Two year', 'Yes', 'Credit card (automatic)', 108.6, '7690.9']
print(model.predict(instance))

['No']


In [3]:
%%writefile setup.py
from setuptools import setup
from setuptools import find_packages

REQUIRED_PACKAGES = ['xgboost']

setup(
    name='custom_predict',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    scripts=['predictor.py'])

Writing setup.py


In [4]:
!python setup.py sdist --formats=gztar

running sdist
running egg_info
creating custom_predict.egg-info
writing custom_predict.egg-info/PKG-INFO
writing dependency_links to custom_predict.egg-info/dependency_links.txt
writing requirements to custom_predict.egg-info/requires.txt
writing top-level names to custom_predict.egg-info/top_level.txt
writing manifest file 'custom_predict.egg-info/SOURCES.txt'
reading manifest file 'custom_predict.egg-info/SOURCES.txt'
writing manifest file 'custom_predict.egg-info/SOURCES.txt'
running check


creating custom_predict-0.1
creating custom_predict-0.1/custom_predict.egg-info
copying files to custom_predict-0.1...
copying README.md -> custom_predict-0.1
copying predictor.py -> custom_predict-0.1
copying setup.py -> custom_predict-0.1
copying custom_predict.egg-info/PKG-INFO -> custom_predict-0.1/custom_predict.egg-info
copying custom_predict.egg-info/SOURCES.txt -> custom_predict-0.1/custom_predict.egg-info
copying custom_predict.egg-info/dependency_links.txt -> custom_predict-0.1/custom_

In [5]:
!gsutil cp ./dist/custom_predict-0.1.tar.gz gs://churn-model-sri/custom_predict-0.1.tar.gz

Copying file://./dist/custom_predict-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.2 KiB/  1.2 KiB]                                                
Operation completed over 1 objects/1.2 KiB.                                      


In [6]:
!gcloud beta ai-platform models create ChurnPredictor --regions us-central1 --enable-console-logging

Using endpoint [https://ml.googleapis.com/]
Created ml engine model [projects/srivatsan-project/models/ChurnPredictor].


In [7]:
! gcloud --quiet beta ai-platform versions create V1 \
  --model ChurnPredictor \
  --runtime-version 2.3 \
  --python-version 3.7 \
  --origin gs://churn-model-sri/ \
  --package-uris gs://churn-model-sri/custom_predict-0.1.tar.gz \
  --prediction-class predictor.ChurnPredictor 

Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


In [ ]:
#{"instances": ["7317-GGVPB", "Male", 0, "Yes", "No", 71, "Yes", "Yes", "Fiber optic", "No", "Yes", "Yes", "Yes", "Yes", "Yes", "Two year", "Yes", "Credit card (automatic)", 108.6, "7690.9"]}

In [8]:
!gcloud ai-platform predict --model ChurnPredictor --version V1 --json-request input.json

Using endpoint [https://ml.googleapis.com/]
['No']
